## Focusing on terse `FeatNets` training/eval
9.28.2020

In [4]:
import os, sys
import copy as copyroot
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from fastai2.basics import *
from fastai2.vision.all import *
%load_ext autoreload
%autoreload 2

from module.mnist_helpers import build_df, eda_fig_1, build_dls
from module.mnist_models import FeatsNet
from module.mnist_metrics import metrics_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dls_tl = build_dls(target='topleft')
dls_cr = build_dls(target='center')

### Grid Fit

In [7]:
EPOCHS = 20
TMP_FN = 'assets/tmp.csv'

feats = [
         ('pix', 'pts11'),
         ('pix', 'pts12'),
         ('pix', 'pts22'),
         ('', 'pts11'),
         ('', 'pts12'),
         ('', 'pts22'),
        ]

dls = [dls_tl, dls_cr]

list_df_metrics = []

t0 = time.time()

for dl_ind in (0,1):
    for _feat in feats:

        featsnet  = FeatsNet(feats=_feat)
        tmplogger = CSVLogger(fname=TMP_FN)
        learn     = Learner(dls[dl_ind], 
                            featsnet, 
                            loss_func=MSELossFlat(), 
                            cbs=[tmplogger])

        with learn.no_logging(): learn.fit(EPOCHS)

        _df = metrics_df(
                   learn, 
                   s_model =    f'FeatsNet1.{_feat[-1][-2:]}',
                   s_details =  f'feats={",".join(_feat)}',
                   s_target =   "topleft" if dl_ind == 0 else "center",
                  )
        
        list_df_metrics.append(_df)
        
print(f'done in {round(time.time() - t0,0)} secs ')
print(f'{len(list_df_metrics)} metric_dfs built')

done in 578.0 secs 
12 metric_dfs built


In [9]:
df = pd.concat(list_df_metrics)
df.reset_index(inplace=True, drop=True)
df.sort_values(by='mse', ascending=True)

,model,details,target,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
15,FeatsNet1.12,"feats=pix,pts12",center,train,0.000644,0.019390,0.881692,0.030806,0.704996,0.001288,0.902266
17,FeatsNet1.22,"feats=pix,pts22",center,train,0.000721,0.020405,0.866515,0.032513,0.688643,0.001441,0.890594
13,FeatsNet1.11,"feats=pix,pts11",center,train,0.000828,0.021696,0.843753,0.034869,0.666083,0.001656,0.874317
14,FeatsNet1.12,"feats=pix,pts12",center,valid,0.001439,0.027464,0.701482,0.044069,0.562362,0.002877,0.767991
16,FeatsNet1.22,"feats=pix,pts22",center,valid,0.001543,0.028703,0.682301,0.045673,0.546433,0.003086,0.751183
12,FeatsNet1.11,"feats=pix,pts11",center,valid,0.001619,0.028959,0.657668,0.046624,0.536988,0.003237,0.738954
20,FeatsNet1.12,"feats=,pts12",center,valid,0.002431,0.038564,0.543687,0.060137,0.402784,0.004861,0.608005
21,FeatsNet1.12,"feats=,pts12",center,train,0.002500,0.039875,0.563935,0.062550,0.401000,0.005000,0.620444
22,FeatsNet1.22,"feats=,pts22",center,valid,0.003556,0.048313,0.357758,0.075117,0.254029,0.007112,0.426517
23,FeatsNet1.22,"feats=,pts22",center,train,0.003979,0.052142,0.350085,0.080105,0.232893,0.007958,0.395938


In [10]:
df.to_csv('assets/metrics-dfs/rcn6b-featsnet1x.csv')